In [1]:
import tensorflow as tf
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
from sklearn.datasets import load_iris
from sklearn import preprocessing

In [2]:
# Import dataset with panda
df = pd.read_csv('output.csv')
features_considered = ['embb_req', 'embb_true', 'urllc_req', 'urllc_true','miot_req', 'miot_true','mmtc_req', 'mmtc_true','voice_req', 'voice_true']
features = df[features_considered]
features.index = df['round']
features.head()


,embb_req,embb_true,urllc_req,urllc_true,miot_req,miot_true,mmtc_req,mmtc_true,voice_req,voice_true
round,,,,,,,,,,
0,43416984823,17392987032,662474056,662474056,341195262,341195262,349262371,349262371,597551580,597551580
1,67809055201,17761759260,551875687,551875687,298211255,298211255,370954027,370954027,951136126,951136126
2,67756318810,17713912304,609694449,609694449,336192886,336192886,289397074,289397074,920173016,920173016
3,57100587829,18946544273,30189256,30189256,42895685,42895685,48573808,48573808,722280280,722280280
4,46781070284,18530565474,194927480,194927480,120669539,120669539,135742773,135742773,518151162,518151162


In [3]:
# Parametes 
TRAIN_SPLIT = 64000
tf.random.set_seed(13)
past_history = 1
future_target = 0
step = 1
BATCH_SIZE = 32 # combines consecutive elements of this dataset into batches.
BUFFER_SIZE = 80000 # >= len dataset for perfetct shuffling
EVALUATION_INTERVAL = 2000 # steps_per_epoch = TotalTrainingSamples / TrainingBatchSize
VALIDATION_STEPS = 500 # validation_steps = TotalvalidationSamples / ValidationBatchSize
EPOCHS = 20
bs_capacity = 20000000000


In [4]:
# Normalize
dataset = features.values
dataset = dataset / bs_capacity


In [5]:
def multivariate_data(dataset, target, start_index, end_index, history_size, target_size, step):
  data = []
  labels = []

  start_index = start_index + history_size
  if end_index is None:
    end_index = len(dataset) - target_size

  for i in range(start_index, end_index):
    indices = range(i-history_size, i, step)

    data.append(dataset[indices])
    labels.append(target[i+target_size])

  return np.array(data), np.array(labels)


In [6]:
# Create validation and training set
x_train, y_train = multivariate_data(dataset[1:, (0, 2, 4, 6, 8)], dataset[1:, (1, 3, 5, 7, 9)], 0, TRAIN_SPLIT, past_history,future_target, step)
x_val, y_val = multivariate_data(dataset[1:, (0, 2, 4, 6, 8)], dataset[1:, (1, 3, 5, 7, 9)], TRAIN_SPLIT, None, past_history, future_target, step)

train_data = tf.data.Dataset.from_tensor_slices((x_train, y_train))
train_data = train_data.cache().shuffle(BUFFER_SIZE).batch(BATCH_SIZE).repeat()

val_data = tf.data.Dataset.from_tensor_slices((x_val, y_val))
val_data = val_data.batch(BATCH_SIZE).repeat()


In [7]:
# Create RNN
model = tf.keras.models.Sequential()
model.add(tf.keras.layers.LSTM(256, input_shape=x_train.shape[-2:])) # 4*(len features input * len output + input^2)
model.add(tf.keras.layers.Dense(5))

# Compile RNN
model.compile(optimizer=tf.keras.optimizers.RMSprop(), loss='mae')

# Train RNN
history = model.fit(train_data, epochs=EPOCHS, steps_per_epoch=EVALUATION_INTERVAL, validation_data=val_data, validation_steps=VALIDATION_STEPS)


Train for 2000 steps, validate for 500 steps
Epoch 1/20
2000/2000 [==============================] - 13s 6ms/step - loss: 0.0189 - val_loss: 0.0150
Epoch 2/20
2000/2000 [==============================] - 11s 5ms/step - loss: 0.0165 - val_loss: 0.0130
Epoch 3/20
2000/2000 [==============================] - 11s 5ms/step - loss: 0.0162 - val_loss: 0.0133
Epoch 4/20
2000/2000 [==============================] - 11s 5ms/step - loss: 0.0160 - val_loss: 0.0136
Epoch 5/20
2000/2000 [==============================] - 11s 5ms/step - loss: 0.0160 - val_loss: 0.0140
Epoch 6/20
2000/2000 [==============================] - 11s 5ms/step - loss: 0.0159 - val_loss: 0.0128
Epoch 7/20
2000/2000 [==============================] - 11s 5ms/step - loss: 0.0159 - val_loss: 0.0129
Epoch 8/20
2000/2000 [==============================] - 11s 5ms/step - loss: 0.0158 - val_loss: 0.0126
Epoch 9/20
2000/2000 [==============================] - 11s 5ms/step - loss: 0.0158 - val_loss: 0.0136
Epoch 10/20
2000/2000 [=====

In [8]:
#Example prediction
prediction = model.predict(x_val, batch_size=BATCH_SIZE)[0]

print(prediction)
print(y_val[0])

# Prediction with new input value
test = [[[28117743919, 554516117, 283582751, 309281138, 510198997]]]
test = np.array(test)/bs_capacity

prediction = model.predict(test)
print(prediction[0])
print(y_train[2]) # because I took the input of the first values in x_train for test


[0.63437986 0.01657911 0.00797648 0.00768435 0.01096512]
[0.5927385  0.00791548 0.00411664 0.00366136 0.0112927 ]
[0.76417774 0.00996409 0.00489848 0.0038805  0.01849279]
[0.92652827 0.00974637 0.00603348 0.00678714 0.02590756]


In [9]:
# Save trained model
model.save("model.h5")
